# Fig. 8: Mean Opacities with fit distributions

In [ ]:
%run header.py

In [ ]:
d      = np.load('data/default_opacities_smooth.npz')
a      = d['a']
gsca   = d['g'].T
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
# define temperature and wavelength arrays

T_array = np.logspace(0, np.log10(1500), 250)
nu = c.c.cgs.value / lam

# calculate the planck function and their derivatives for all those temperatures and wavelength

Bnu    = np.array([aux.planck_B_nu(nu, T) for T in T_array])
dBnudT = np.array([aux.planck_dBnu_dT(nu, T) for T in T_array])

## Calculate mean opacities for all temperatures

For the Rosseland Mean, we need the *extinction opacity*

$$
\kappa_\nu^\mathrm{ext}(a) = \kappa_\nu^{abs}(a) + (1-g)\,\kappa_\nu^{sca}(a)
$$

then the total opacity of the size distribution is given by

$$
\kappa_\nu^\mathrm{abs,tot} = \frac{\int_0^{a_\mathrm{max}} n(a)\,m\,\kappa_\nu^\mathrm{abs}(a) \,\mathrm{d}a}{\int_0^{a_\mathrm{max}} n(a)\,m \,\mathrm{d}a}
$$

The Planck mean opacity is 

$$
\bar \kappa_\mathrm{P}(T) = \frac{\int_0^\infty \kappa_\nu^\mathrm{abs,tot}\, B_\nu(T)\,\mathrm{d}\nu}{\int_0^\infty  B_\nu(T)\,\mathrm{d}\nu}
$$

and the Rosseland mean opacity is

$$
\bar \kappa_\mathrm{R}(T) = \left( \frac{\int_0^\infty \frac{1}{\kappa_\nu^\mathrm{ext,tot}} \, \frac{\mathrm{d}B_\nu(T)}{\mathrm{d}T}\,\mathrm{d}\nu}{\int_0^\infty  \frac{\mathrm{d}B_\nu(T)}{\mathrm{d}T}\,\mathrm{d}\nu}\right)^{-1}
$$

In [ ]:
k_ext = k_abs + (1 - gsca) * k_sca

In [ ]:
q = 3.5
amax = 0.1
power_law = a**(4 - q)
power_law[a > amax] = 0
power_law = power_law / power_law.sum()

fits1 = np.zeros([len(T_array), len(a)])
fits2 = np.zeros([len(T_array), len(a)])

k_P_p  = np.zeros_like(T_array)
k_P_f1 = np.zeros_like(T_array)
k_P_f2 = np.zeros_like(T_array)
k_R_p  = np.zeros_like(T_array)
k_R_f1 = np.zeros_like(T_array)
k_R_f2 = np.zeros_like(T_array)

# for each temperature ...

for it, T in enumerate(T_array):
    
    # get the size distribution fit number 1
    
    f1 = opacity.get_B11_fit(T, a)
    f1 = f1 / f1.sum()    
    fits1[it, :] = f1
    
    # get the size distribution fit number 1
    
    f2 = opacity.get_B11S_fit(T, a)[0]
    f2 = (f2 * a) / (f2 * a).sum()
    fits2[it, :] = f2
    
    # sum the absorption opacity
    
    k_abs_p = (k_abs * power_law[None, :]).sum(1)
    k_abs_f1 = (k_abs * f1[None, :]).sum(1)
    k_abs_f2 = (k_abs * f2[None, :]).sum(1)
    
    # sum the EXTINCTION opacity
    
    k_ext_p = (k_ext * power_law[None, :]).sum(1)
    k_ext_f1 = (k_ext * f1[None, :]).sum(1)
    k_ext_f2 = (k_ext * f2[None, :]).sum(1)
    
    # calculate Planck opacity for the fit and the power-law
    
    B   = np.trapz(Bnu[it, :], x=nu)
    k_P_p[it]  = np.trapz(Bnu[it, :] * k_abs_p, x=nu) / B
    k_P_f1[it] = np.trapz(Bnu[it, :] * k_abs_f1, x=nu) / B
    k_P_f2[it] = np.trapz(Bnu[it, :] * k_abs_f2, x=nu) / B
    
    # calculate Rosseland opacity for the fit and the power-law
    
    B   = np.trapz(dBnudT[it, :], x=nu)
    k_R_p[it]  = B / np.trapz(dBnudT[it, :] / k_ext_p, x=nu)
    k_R_f1[it] = B / np.trapz(dBnudT[it, :] / k_ext_f1, x=nu)
    k_R_f2[it] = B / np.trapz(dBnudT[it, :] / k_ext_f2, x=nu)

In [ ]:
try:
    from widget import plotter
    plotter(a, fits1, data2=fits2, xlog=True, ylog=True, ylim=[1e-10,2], times=T_array)
except ImportError:
    print('widget not installed')

#### The spline fit to one of the curves

In [ ]:
from scipy.interpolate import LSQUnivariateSpline, BSpline

k = 2 # order of the spline
positions = [0.5, 1, 2, 2.5, 3] # where to set the points

# create a spline fit in log-log space

s1 = LSQUnivariateSpline(np.log10(T_array), np.log10(k_P_f2), positions, k=k)

# get the knots and coefficients. the "ghost" values are removed

knots = s1.get_knots()
knots = k*[knots[0]] + list(knots) + k*[knots[-1]]
coeff = s1.get_coeffs()

print('knots        = '+ ', '.join(['{:6.3g}'.format(_k) for _k in knots]))
print('coefficients = '+ ', '.join(['{:6.3g}'.format(_k) for _k in coeff]))

# create a spline base on those knots 

s2 = BSpline(knots, coeff, k)

# plot it

plt.loglog(T_array, k_P_f2)
plt.loglog(T_array, 10**s2(np.log10(T_array)), '--')

#### Load pre-calculated opacity ranges

In [ ]:
with np.load('kPR1.npz') as fid:
    k_P_f1_array = fid['k_P_f1_array']
    k_R_f1_array = fid['k_R_f1_array']
    T_grid       = fid['T_array']
    
kP1min = k_P_f1_array.copy()
kP1max = k_P_f1_array.copy()
kR1min = k_R_f1_array.copy()
kR1max = k_R_f1_array.copy()
for i in range(k_P_f1_array.ndim - 1):
    kP1min = kP1min.min(-1)
    kP1max = kP1max.max(-1)

    kR1max = kR1max.max(-1)
    kR1min = kR1min.min(-1)

#### Do the plot

In [ ]:
f, ax = plt.subplots(figsize=(3.5, 3.5 * 0.65))
lP1, = ax.loglog(T_array, k_P_p, label='Planck mean, power-law')
lP2, = ax.loglog(T_array, k_P_f1, '--', c=lP1.get_color(), label='Planck mean, B11')
lP3, = ax.loglog(T_array, k_P_f2, ':',  c=lP1.get_color(), label='Planck mean, B15')
lP4  = ax.fill_between(T_grid, kP1min, kP1max,  facecolor=lP1.get_color(), label='B11S, Planck mean', alpha=0.35)

lR1, = ax.loglog(T_array, k_R_p, label='Rosseland mean, power-law')
lR2, = ax.loglog(T_array, k_R_f1, '--', c=lR1.get_color(), label='Rosseland mean, B11')
lR3, = ax.loglog(T_array, k_R_f2, ':',  c=lR1.get_color(), label='Rosseland mean, B15')
lR4  = ax.fill_between(T_grid, kR1min, kR1max,  facecolor=lR1.get_color(), label='B11S, Rosseland mean', alpha=0.35)

#ax.loglog(T_array, 10.**(s2(np.log10(T_array))), 'k--', alpha=0.5)

dummies = []
dummies += ax.plot([], [], 'k-',  label='power-law')
dummies += ax.plot([], [], 'k--', label='B11')
dummies += ax.plot([], [], 'k:',  label='B11S')
dummies += ax.plot([], [], c=lP1.get_color(),  label='Planck mean')
dummies += ax.plot([], [], c=lR1.get_color(),  label='Rosseland mean')
dummies += [lP4, lR4]

ax.legend(dummies, [d.get_label() for d in dummies], fontsize='xx-small')
#ax.legend(fontsize='xx-small')

ax.set_xlabel('$T\,$ [K]')
ax.set_ylabel(r'$\bar\kappa$ [cm$^2$/g]')

f.subplots_adjust(
    left=0.14, right=0.98,
    bottom=0.17, top=0.97
    )
f.savefig('../figures/fig8_mean_opacities.pdf')